In [1]:
%cd /Users/maimai/Desktop/project/May

import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
import os
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import multiprocessing
from multiprocessing import Pool
from IPython.display import display

/Users/maimai/Desktop/project/May
('Reading config file from location: '
 '/Users/maimai/Desktop/project/May/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/Users/maimai/Desktop/project/May/followers',
                    'friends': '/Users/maimai/Desktop/project/May/friends'},
          'cwd': '/Users/maimai/Desktop/project/May/data/givenchy',
          'ml': '/Users/maimai/Desktop/project/May/data/givenchy/pickle',
          'networkx': {'all': '/Users/maimai/Desktop/project/May/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/Users/maimai/Desktop/project/May/data/givenchy/pic

# Load data: unique_users

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

path = "/Users/maimai/Desktop/project/May/data/givenchy/pickle"  

def write_to_pickle(file_path, data):

    bytes_out = pickle.dumps(data)
    with open(file_path, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    
    return pickle.loads(bytes_in)


In [3]:
unique_users = load_pickle(path+'/users.dat')

# process data

In [4]:
# intervals = [60, 30, 15, 7]
intervals = [30]

# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y
    
# infectedNetworkStats = {}
# for interval in intervals:
#     statsDictionary = {}
#     for timestamp in range(0, 24*60+1, interval):
#         infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
#         statsDictionary[timestamp] = {}
#         statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
#         statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
#         statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
#     infectedNetworkStats[interval] = statsDictionary

def infect_totalNodeInfected(users):
    infected_dataframe = users[users.TwM_tCurrent>=0]
    states = infected_dataframe.shape[0]
    return states

def infect_totalFollowers(users):
    infected_dataframe = users[users.TwM_tCurrent>=0]
    states = sum(infected_dataframe.UsM_followersCount)
    return states

def infect_totalFriends(users):
    infected_dataframe = users[users.TwM_tCurrent>=0]
    states = sum(infected_dataframe.UsM_friendsCount)
    return states

in_degree = list(unique_users.friends_count)
out_degree = list(unique_users.followers_count)
degree = in_degree + out_degree

In [6]:
users = pd.DataFrame(columns=
                    ['label'  ,
        # UsM: User metadata
        'UsM_deltaDays'  ,
        'UsM_statusesCount'  ,
        'UsM_followersCount'  ,
        'UsM_favouritesCount'  ,
        'UsM_friendsCount'  ,
        'UsM_listedCount'  ,
        'UsM_normalizedUserStatusesCount'  ,
        'UsM_normalizedUserFollowersCount'  ,
        'UsM_normalizedUserFavouritesCount'  ,
        'UsM_normalizedUserListedCount'  ,
        'UsM_normalizedUserFriendsCount'  ,          
        'UsM_deltaDays0'  ,
        'UsM_statusesCount0'  ,
        'UsM_followersCount0'  ,
        'UsM_favouritesCount0'  ,
        'UsM_friendsCount0'  ,
        'UsM_listedCount0'  ,
        'UsM_normalizedUserStatusesCount0'  ,
        'UsM_normalizedUserFollowersCount0'  ,
        'UsM_normalizedUserFavouritesCount0'  ,
        'UsM_normalizedUserListedCount0'  ,
        'UsM_normalizedUserFriendsCount0'  ,
        'UsM_deltaDays-1'  ,
        'UsM_statusesCount-1'  ,
        'UsM_followersCount-1'  ,
        'UsM_favouritesCount-1'  ,
        'UsM_friendsCount-1'  ,
        'UsM_listedCount-1'  ,
        'UsM_normalizedUserStatusesCount-1'  ,
        'UsM_normalizedUserFollowersCount-1'  ,
        'UsM_normalizedUserFavouritesCount-1'  ,
        'UsM_normalizedUserListedCount-1'  ,
        'UsM_normalizedUserFriendsCount-1'  ,
        # TwM: Tweet metadata
        'TwM_t0'  ,
        'TwM_tSeed0'  ,
        'TwM_t-1'  ,
        'TwM_tSeed-1'  ,
        'TwM_tCurrent'  ,
        # Nw: Network
        'Nw_degree'  ,
        'Nw_inDegree'  ,
        'Nw_outDegree'  ,
        'Nw_degree0'  ,
        'Nw_inDegree0'  ,
        'Nw_outDegree0'  ,
        'Nw_degree-1'  ,
        'Nw_inDegree-1'  ,
        'Nw_outDegree-1'  ,
        'Nw_degreeSeed0'  ,
        'Nw_inDegreeSeed0'  ,
        'Nw_outDegreeSeed0'  ,
        'Nw_degreeSeed-1'  ,
        'Nw_inDegreeSeed-1'  ,
        'Nw_outDegreeSeed-1'  ,
        # SNw: Spreading Network
        'SNw_nFriendsInfected'  ,
        'SNw_friendsInfectedRatio'  ,
        # 加一行generation，for 0&-1相对应的值
        'SNw_generation',
        'SNw_generation0'  ,
        'SNw_generation-1'  ,
        'SNw_timeSinceSeed0'  ,
        'SNw_timeSinceSeed-1'  ,
        'SNw_totalNodesInfected'  ,
        'SNw_nodeInfectedCentrality'  ,
        'SNw_totalInDegree'  ,
        'SNw_totalOutDegree'  ,
        'SNw_inDegreeCentrality'  ,
        'SNw_inDegreeCentrality0'  ,
        'SNw_inDegreeCentrality-1'  ,
        'SNw_outDegreeCentrality'  ,
        'SNw_outDegreeCentrality0'  ,
        'SNw_outDegreeCentrality-1'  ,
        'SNw_inDegreeCentralitySeed0',
        'SNw_outDegreeCentralitySeed0',
        'SNw_inDegreeCentralitySeed-1',
        'SNw_outDegreeCentralitySeed-1',
        # Stat: Statistical
        'Stat_average_kOut'  ,
        'Stat_average_t'  ,
        'Stat_average_deltaDays'  ,
        'Stat_average_statusesCount'  ,
        'Stat_average_followersCount'  ,
        'Stat_average_favouritesCount'  ,
        'Stat_average_friendsCount'  ,
        'Stat_average_listedCount'  ,
        'Stat_average_normalizedUserStatusesCount'  ,
        'Stat_average_normalizedUserFollowersCount'  ,
        'Stat_average_normalizedUserFavouritesCount'  ,
        'Stat_average_normalizedUserListedCount'  ,
        'Stat_average_normalizedUserFriendsCount'  ,                
        'Stat_max_kOut'  ,
        'Stat_min_kOut'  
                   ]) 
users['user_id']=unique_users['user_id']

users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5973 entries, 0 to 6013
Data columns (total 91 columns):
label                                         0 non-null object
UsM_deltaDays                                 0 non-null object
UsM_statusesCount                             0 non-null object
UsM_followersCount                            0 non-null object
UsM_favouritesCount                           0 non-null object
UsM_friendsCount                              0 non-null object
UsM_listedCount                               0 non-null object
UsM_normalizedUserStatusesCount               0 non-null object
UsM_normalizedUserFollowersCount              0 non-null object
UsM_normalizedUserFavouritesCount             0 non-null object
UsM_normalizedUserListedCount                 0 non-null object
UsM_normalizedUserFriendsCount                0 non-null object
UsM_deltaDays0                                0 non-null object
UsM_statusesCount0                            0 non-null obje

In [7]:
for i in range(len(unique_users)):
    source_candidates = unique_users.loc[i,'source_candidates']
    sources = [x for x in source_candidates]
    sources_dataframe = unique_users.iloc[sources]         
                    
                    
    #degreeList = degreeList = list(degree[i] for i in sources)
    degreeList = list(unique_users.loc[i, 'followers_count'] + unique_users.loc[i, 'friends_count']  for i in sources)

    print(i)

    
    users.loc[i,'UsM_deltaDays']=unique_users.loc[i,'user_created_days']
    users.loc[i,'UsM_statusesCount']=unique_users.loc[i,'user_statuses_count']
    users.loc[i,'UsM_followersCount']=unique_users.loc[i,'followers_count']
    users.loc[i,'UsM_favouritesCount']=unique_users.loc[i,'user_favourites_count']
    users.loc[i,'UsM_friendsCount']=unique_users.loc[i,'friends_count']
    users.loc[i,'UsM_listedCount']=unique_users.loc[i,'user_listed_count']
    users.loc[i,'UsM_normalizedUserStatusesCount']=unique_users.loc[i,'normalized_user_statuses_count']
    users.loc[i,'UsM_normalizedUserFollowersCount']=unique_users.loc[i,'normalized_user_followers_count']
    users.loc[i,'UsM_normalizedUserFavouritesCount']=unique_users.loc[i,'normalized_user_favourites_count']
    users.loc[i,'UsM_normalizedUserListedCount']=unique_users.loc[i,'normalized_user_listed_count']
    users.loc[i,'UsM_normalizedUserFriendsCount']=unique_users.loc[i,'normalized_user_friends_count']
    
    users.loc[i,'Nw_degree']= degree[i]
    users.loc[i,'Nw_inDegree']= in_degree[i]
    users.loc[i,'Nw_outDegree']= out_degree[i]

    users.loc[i,'SNw_nFriendsInfected']=len(sources)
    users.loc[i,'SNw_friendsInfectedRatio']=safe_division(len(sources), unique_users.loc[i,'friends_count'])
   
    users.loc[i,'Stat_average_kOut']=round(mean(degreeList), 1)
    users.loc[i,'Stat_average_deltaDays']=sources_dataframe.user_created_days.mean()
    users.loc[i,'Stat_average_statusesCount']=sources_dataframe.user_statuses_count.mean()
    users.loc[i,'Stat_average_followersCount']=sources_dataframe.followers_count.mean()
    users.loc[i,'Stat_average_favouritesCount']=sources_dataframe.user_favourites_count.mean()
    users.loc[i,'Stat_average_friendsCount']=sources_dataframe.friends_count.mean()
    users.loc[i,'Stat_average_listedCount']=sources_dataframe.user_listed_count.mean()
    users.loc[i,'Stat_average_normalizedUserStatusesCount']=sources_dataframe.normalized_user_statuses_count.mean()
    users.loc[i,'Stat_average_normalizedUserFollowersCount']=sources_dataframe.normalized_user_followers_count.mean()
    users.loc[i,'Stat_average_normalizedUserFavouritesCount']=sources_dataframe.normalized_user_favourites_count.mean()
    users.loc[i,'Stat_average_normalizedUserListedCount']=sources_dataframe.normalized_user_listed_count.mean()
    users.loc[i,'Stat_average_normalizedUserFriendsCount']=sources_dataframe.normalized_user_friends_count.mean()
    if len(degreeList)!=0 :
        users.loc[i,'Stat_max_kOut']=max(degreeList) 
        users.loc[i,'Stat_min_kOut']=min(degreeList) 
    else:
        users.loc[i,'Stat_max_kOut']=''
        users.loc[i,'Stat_min_kOut']=''
users.info()

0


KeyError: 'the label [user_statuses_count] is not in the [index]'

In [ ]:
write_to_pickle(path+'/features.pkl', users)
users=load_pickle('features.pkl')
# users